In [ ]:
# general
import os
import uuid
import numpy as np
import pandas as pd
import torch
from datetime import datetime as dt, timedelta
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
from tqdm import tqdm
import matplotlib.pyplot as plt
from src.misc import *
from src.io_utils import write_image, concatenate_rasters
from src.cloud_detector import CloudDetector
from src.datasets import load_landslide_df
from empatches import EMPatches

# geospatial packages
import geopandas as gpd
import shapely
from shapely.geometry import shape
import fiona
import richdem
from pyproj import CRS

# planetary computer and related
import planetary_computer as pc
import pystac_client as stac
import stackstac as ss
import distributed as dask
import rasterio as rio
import rioxarray as rxr

IMAGE_FOLDER = 'indonesia'
S2_NATIVE_CRS = 32750

In [ ]:
# read inventory
gdf = gpd.read_file('inventories/indonesia_landslides.gpkg')

# infer mapping extent (as a convex hull)
aoi = gdf.union_all().convex_hull
# buffer it by at least 500m in each direction
buffer = max(find_buffer_values(aoi, meters=500))
aoi = aoi.buffer(buffer, cap_style='square', join_style='mitre', mitre_limit=5.0)

# 1. Search & download S2 data

### Search

In [ ]:
# open the STAC catalog
catalog = stac.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier = pc.sign_inplace,
)

# get only products from 90 days before to 30 days after activation date
event_date = gdf['event_date'].iloc[0]
min_pre_date = (event_date + timedelta(days=-90)).strftime('%Y-%m-%d')
max_pre_date = (event_date + timedelta(seconds=-1)).strftime('%Y-%m-%d')
min_post_date = event_date.strftime('%Y-%m-%d')
max_post_date = (event_date + timedelta(days=30, seconds=-1)).strftime('%Y-%m-%d')

# define queries
search = {
    "pre_s2"  :  {
        "collections" :  ["sentinel-2-l2a"],
        "intersects"  :  aoi,
        "datetime"    :  f"{min_pre_date}/{max_pre_date}",
        "query"       :  {"eo:cloud_cover": {"lt": 90}}
    },
    "post_s2"  :  {
        "collections" :  ["sentinel-2-l2a"],
        "intersects"  :  aoi,
        "datetime"    :  f"{min_post_date}/{max_post_date}",
        "query"       :  {"eo:cloud_cover": {"lt": 90}}
    },
    "dem"  :  {
        "collections" :  ["alos-dem"],
        "intersects"  :  aoi,
        "datetime"    :  "2016-12-07/2016-12-08",
        "query"       :  {}
    },
}

# execute queries
items = {}
for k in search.keys():
    items[k] = catalog.search(**search[k]).item_collection()
    print(f"[{k.upper()}]: {len(items[k])}", end=" | " if k != "dem" else "\n", )


### Download

In [ ]:
# initialize Dask client
client = dask.Client(processes=False)
print(f"Dask dashboard: {client.dashboard_link}")

assets = {
    's2' : ['AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP']
}
# NB: 'visual' and 'rendered_preview' cannot be included because they are multi-band assets
# rio.to_raster() can only handle 1-band assets

# download the corresponding Sentinel-2 products
print(f"Downloading products...")
for k in items.keys():
    
    sat = k.split('_')[-1]

    # select collection of products for landslide i and date-range_satellite k
    item_collection = items[k]

    if sat in ['s1', 's2']:

        # download products
        # NB: 2+ products with the same date are really contiguous tiles of the
        # same product, so when referring to an 'item' here we are actually
        # referring to an item collection (1+) of contiguous tiles. So we need
        # to split the item collection by date.
        for item in tqdm(split_item_collection_by_date(item_collection), desc=k):

            acquisition_date = item[0].datetime.strftime('%Y-%m-%dT%H%M%SZ')
            #epsg = item[0].properties['proj:epsg']  # set CRS to the first contiguous tile (for disambiguation)

            orbit_state = ''
            if sat == 's1':
                orbit_state = '_asc' if item[0].properties['sat:orbit_state'] == 'ascending' else '_desc'
                
            cloud_cover = ''
            if sat == 's2':
                cloud_cover = f"_cc{round(item[0].properties['eo:cloud_cover'])}"
            
            # set output filename
            fn = f"images/{IMAGE_FOLDER}/{k.split('_')[0]}/{acquisition_date}{orbit_state}{cloud_cover}.tif"
            if os.path.exists(fn): continue
            os.makedirs(os.path.dirname(fn), exist_ok=True)

            # cast item from pystac.item.Item to xarray.DataArray
            s = ss.stack(
                item,
                bounds_latlon = aoi.bounds,
                assets = assets[sat],
                resolution = 10,
                epsg = S2_NATIVE_CRS,
                xy_coords = 'center',
                #dtype = np.uint16, # set later
                #fill_value = 0, # later we will set nodata value as 0
                )
            
            # if processing baseline >= 4 (introduced in 25 january 2022),
            # harmonize data to the old processing baseline
            s = harmonize_to_old_processing_baseline(s)
            
            # cast item from xarray.DataArray to xarray.Dataset
            #s = s.to_dataset(dim='band')

            # cast SCL to uint8 (useless when rasterizing, because
            # rio.to_raster() saves every band with the dtype of the first band)
            #s['SCL'] = s['SCL'].astype('uint8')

            # stack eventual overlapping tiles by performing a median composite
            # in the time dimension (ignore NaNs)
            s = s.median(dim='time', keep_attrs=True, skipna=True)
            
            # cast to uint16 (this also fills NaNs with 0)
            s = s.round().astype(np.uint16)

            # set nodata value as 0
            s = s.rio.write_nodata(0)

            # save to geotiff
            s.rio.to_raster(fn, compress='lzw')

            # change band names
            # (this is needed because rio.to_raster doesn't save band names in
            # the geotiff by default)
            with rio.open(fn, 'r+') as ds:
                ds.descriptions = assets[sat]
            
            # NB: another option would be to convert the xarray.DataArray to a xarray.Dataset:
            # s = s.to_dataset("band")
            # and then drop the band dimension (as bands are now variables):
            # s = s.squeeze("band", drop=True)
                
    elif sat == 'dem':
        for item in tqdm(split_item_collection_by_date(item_collection), desc=k):

            # set output filename
            fn = f"images/{IMAGE_FOLDER}/dem_30m.tif"
            if os.path.exists(fn): continue
            os.makedirs(os.path.dirname(fn), exist_ok=True)
            
            # cast item from pystac.item.Item to xarray.DataArray
            s = ss.stack(
                item,
                bounds_latlon = aoi.bounds,
                assets = ['data'],
                #resolution = 10,   # USE DEFAULT AND RESIZE AFTER COMPUTING SLOPE AND ASPECT
                epsg = S2_NATIVE_CRS,
                xy_coords = 'center',
                )
            
            # stack eventual overlapping tiles by performing a median composite
            # in the time dimension
            s = s.median(dim='time', keep_attrs=True, skipna=True)

            # fill NaNs with -9999
            s = s.fillna(-9999)
            
            # cast to int
            s = s.round().astype(int)

            # set nodata value as -9999
            s = s.rio.write_nodata(-9999)

            # save to geotiff
            s.rio.to_raster(fn, compress='lzw')

            # change band names
            with rio.open(fn, 'r+') as ds:
                ds.descriptions = ['data']


# 2. Cloud detection

In [ ]:
cloud_detector = CloudDetector(device='cuda' if torch.cuda.is_available() else 'cpu')

files = nglob(f'images/{IMAGE_FOLDER}/*/*.tif')

for fn in tqdm(files):

    # detect clouds
    with rxr.open_rasterio(fn) as stack:
        if 'cloud_mask' in stack.long_name: continue
        mask = cloud_detector(stack)

    # append cloud mask to the raster
    temp_fn = f'.temp/{str(uuid.uuid4().hex)}.tif'
    write_image(mask, temp_fn, georef_like=fn, compress='lzw', descriptions=['cloud_mask'])
    concatenate_rasters([fn,temp_fn], fn)
    os.remove(temp_fn)

# 3. Generate slope and aspect from DEM

In [ ]:
# https://www.earthdatascience.org/tutorials/get-slope-aspect-from-digital-elevation-model/

dem_xr = rxr.open_rasterio(f'images/{IMAGE_FOLDER}/dem_30m.tif')
dem = dem_xr.data[0]    # to numpy array
dem = richdem.rdarray(dem, no_data=-9999)   # to richdem array

# get geotransform matrix in the richdem format
# GT(0) x-coordinate of the upper-left corner of the upper-left pixel.
# GT(1) w-e pixel resolution / pixel width.
# GT(2) row rotation (typically zero).
# GT(3) y-coordinate of the upper-left corner of the upper-left pixel.
# GT(4) column rotation (typically zero).
# GT(5) n-s pixel resolution / pixel height (negative value for a north-up image).
with rio.open(f'images/{IMAGE_FOLDER}/dem_30m.tif') as ds:
    gtr = list(ds.transform)
    gtr = (gtr[2], gtr[0], 0, gtr[5], 0, gtr[4])
    proj = CRS.from_epsg(S2_NATIVE_CRS).to_proj4()

# set richdem array's geotransform matrix and projection (needed for slope and aspect computation)
dem.geotransform = gtr
dem.projection = proj

# compute slope and aspect
slope = richdem.TerrainAttribute(dem, attrib='slope_degrees')
aspect = richdem.TerrainAttribute(dem, attrib='aspect')
# other DEM-derivatives to consider for landslide mapping:
# https://link.springer.com/article/10.1007/s11069-018-3543-1

# to rioxarray's RasterArray
slope_xr = dem_xr.copy(deep=True, data=np.array(slope)[np.newaxis, :, :])
aspect_xr = dem_xr.copy(deep=True, data=np.array(aspect)[np.newaxis, :, :])

# resize to the same grid of S2 images
template_path = nglob(f'images/{IMAGE_FOLDER}/pre/*')[0]
template_xr = rxr.open_rasterio(template_path)
dem_xr = dem_xr.rio.reproject_match(template_xr, resampling=rio.enums.Resampling.bilinear, nodata=-9999)
slope_xr = slope_xr.rio.reproject_match(template_xr, resampling=rio.enums.Resampling.bilinear, nodata=-1)
aspect_xr = aspect_xr.rio.reproject_match(template_xr, resampling=rio.enums.Resampling.nearest, nodata=-1)

# interpolate missing values (may be caused by reprojection)
dem_xr = dem_xr.rio.interpolate_na('nearest')
slope_xr = slope_xr.rio.interpolate_na('nearest')
aspect_xr = aspect_xr.rio.interpolate_na('nearest')

# save to geotiff
dem_xr.rio.to_raster(f'images/{IMAGE_FOLDER}/dem.tif', compress='lzw')
slope_xr.rio.to_raster(f'images/{IMAGE_FOLDER}/slope.tif', compress='lzw')
aspect_xr.rio.to_raster(f'images/{IMAGE_FOLDER}/aspect.tif', compress='lzw')

# visualize
#richdem.rdShow(slope, axes=False, cmap='magma', figsize=(8, 6))
#richdem.rdShow(aspect, axes=False, cmap='jet', figsize=(8, 6))

# 4. Rasterize landslide polygons

In [ ]:
# Change CRS to S2_NATIVE_CRS
gdf = gdf.to_crs(epsg=S2_NATIVE_CRS)

# Get list of landslide polygons
geom = gdf.geometry.to_list()

# Read metadata from template raster
template_path = nglob(f'images/{IMAGE_FOLDER}/pre/*.tif')[0]
with rio.open(template_path) as template:
    template_shape = template.shape
    template_transform = template.transform

# Rasterize vector using the shape and coordinate system of the template raster
#https://github.com/rasterio/rasterio/blob/33a8c9436f47447de333b843f9565d3586983dd2/rasterio/features.py#L182
rasterized_alltouched = rio.features.rasterize(
    geom,
    out_shape = template_shape,
    fill = 0,
    transform = template_transform,
    all_touched = True, # to avoid gaps between contiguous landslides (since they compete for the same cell)
    default_value = 255,
    dtype = np.uint8
)[np.newaxis,...]

rasterized = rio.features.rasterize(
    geom,
    out_shape = template_shape,
    fill = 0,
    transform = template_transform,
    all_touched = False, # no avoidance of gaps between contiguous landslides
    default_value = 255,
    dtype = np.uint8
)[np.newaxis,...]

# Save rasterized vector
write_image(
    rasterized_alltouched,
    path = f"images/{IMAGE_FOLDER}/polygons_alltouched.tif",
    georef_like = template_path,
    compress = 'lzw'
)

write_image(
    rasterized,
    path = f"images/{IMAGE_FOLDER}/polygons.tif",
    georef_like = template_path,
    compress = 'lzw'
)

# change band name
with rio.open(f"images/{IMAGE_FOLDER}/polygons_alltouched.tif", 'r+') as ds:
    ds.descriptions = ('data',)

with rio.open(f"images/{IMAGE_FOLDER}/polygons.tif", 'r+') as ds:
    ds.descriptions = ('data',)

# 5. Divide into patches

In [ ]:
PATCHSIZE = 256
STRIDE = 128
VISIBLE_LANDSLIDE_PIXELS_THRESHOLD = 1
NOT_VISIBLE_CLASSES = [1,2] # 0 = clear sky, 1 = thick clouds, 2 = thin clouds/fog/smoke, 3 = cloud shadow

# load the landslide dataframe
s2_df = load_landslide_df(f'images/{IMAGE_FOLDER}')
s2_pre_df = s2_df[s2_df['time'] == 'pre']
s2_post_df = s2_df[s2_df['time'] == 'post']

# read rasterized polygons
with rio.open(f'images/{IMAGE_FOLDER}/polygons.tif') as polygons:
    polygons_crs = polygons.crs
    polygons_transform = polygons.transform
    polygons_img = polygons.read(1)

# NB: padding is not performed here, so the patches on the lower and right edges
# of the image may overlap more than the specified stride with the penultimate
# patches of the same edges

# extract patches (p_idx is in (h_start, h_end, w_start, w_end) format)
polygons_p, polygons_p_idx = EMPatches().extract_patches(polygons_img, patchsize=256, stride=128)

# keep patches that contain at least VISIBLE_LANDSLIDE_PIXELS_THRESHOLD landslide pixel
polygons_p_idx_with_vis_ls = []
fn_pre_list = []
fn_post_list = []
n_vis_ls_pixels_list = []
perc_cloudy_pixels_list = []

for p, p_idx in tqdm(zip(polygons_p, polygons_p_idx), total=len(polygons_p_idx)):
    h_start, h_end, w_start, w_end = p_idx

    # discard patch if it does not contain any landslide pixels
    ls_mask = (p == 255)
    n_ls_pixels = np.sum(ls_mask)
    if n_ls_pixels == 0:
        continue

    # discard patch if it does not contain any VISIBLE landslide pixels or if it contains nodata values
    for fn_pre in s2_pre_df['img']:
        raster = rxr.open_rasterio(fn_pre)
        raster = raster.assign_coords({'band' : list(raster.long_name)})
        red = raster.sel(band=['B04']).isel(y=slice(h_start, h_end), x=slice(w_start, w_end))
        # if there are 0 values in red band (i.e. nodata) then the patch is discarded
        if np.any(red == 0):
            continue
        cloud_mask_pre = raster.sel(band='cloud_mask').isel(y=slice(h_start, h_end), x=slice(w_start, w_end))
        cloud_mask_pre = np.isin(cloud_mask_pre, NOT_VISIBLE_CLASSES)

        for fn_post in s2_post_df['img']:
            raster = rxr.open_rasterio(fn_post)
            raster = raster.assign_coords({'band' : list(raster.long_name)})
            red = raster.sel(band=['B04']).isel(y=slice(h_start, h_end), x=slice(w_start, w_end))
            # if there are 0 values in red band (i.e. nodata) then the patch is discarded
            if np.any(red == 0):
                continue
            cloud_mask_post = raster.sel(band='cloud_mask').isel(y=slice(h_start, h_end), x=slice(w_start, w_end))
            cloud_mask_post = np.isin(cloud_mask_post, NOT_VISIBLE_CLASSES)

            cloud_mask_bitemporal = cloud_mask_pre | cloud_mask_post
            perc_cloudy_pixels = cloud_mask_bitemporal.sum() / cloud_mask_bitemporal.size

            vis_ls_mask = np.where(ls_mask, ~cloud_mask_bitemporal, False)
            n_vis_ls_pixels = np.sum(vis_ls_mask)
            if n_vis_ls_pixels == 0:
                continue
            
            fn_pre_list.append(fn_pre)
            fn_post_list.append(fn_post)
            polygons_p_idx_with_vis_ls.append(p_idx)
            n_vis_ls_pixels_list.append(n_vis_ls_pixels)
            perc_cloudy_pixels_list.append(perc_cloudy_pixels)

# create dataframe and save to disk
p_df = pd.DataFrame(polygons_p_idx_with_vis_ls, columns=['h_start', 'h_end', 'w_start', 'w_end'])
p_df['fn_pre'] = fn_pre_list
p_df['fn_post'] = fn_post_list
p_df['fn_dem'] = f'images/{IMAGE_FOLDER}/dem.tif'
p_df['fn_slope'] = f'images/{IMAGE_FOLDER}/slope.tif'
p_df['fn_aspect'] = f'images/{IMAGE_FOLDER}/aspect.tif'
p_df['fn_poly'] = f'images/{IMAGE_FOLDER}/polygons.tif'
p_df['landslide_pixels'] = n_vis_ls_pixels_list
p_df['cloud_cover'] = perc_cloudy_pixels_list
csv_path = f'images/{IMAGE_FOLDER}/patch_couples_size{PATCHSIZE}_stride{STRIDE}.csv'
p_df.to_csv(csv_path, index=False)